In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Common Engine Interface

Throughout the different chapters, multiple chess engines using different algorithms will be developed. In this chapter some common code needed by all engines will be developed.

In general, all engines will need to represent a move along with a score. Therefore, a simple data class `ScoredMove` is introduced, which has two attributes `score` and `move`. The `score` attribute has the type `chess.engine.Score` and is able to represent scores in centipawns as well as Mates. By using the `dataclass` decorator with `order=True` python comparison methods, 
such as `__lt__()`, will be auto-generated.
This allows to compare two scored moves or sort a list of scored moves.
The comparison is only based on the score as the move field has `compare=False`. 

In [ ]:
from dataclasses import dataclass, field
import chess
from chess.engine import Score


@dataclass(order=True)
class ScoredMove:
    """Class for representing a move along with a score."""
    score: Score
    move: chess.Move = field(compare=False)

To simplify writing generic code for playing a chess game with different engines,
all engines will implement a common interface. 
The interface `Engine`,
which is inspired by the `chess.engine` interface,
has two functions: `play` and `analyse`.

The `play` function takes the current board `board` as a parameter 
and returns a `chess.engine.PlayResult` object, 
which contains the next move 
and information if the engine offered a draw or resigned. 

The `analyse` function takes the board `board` as a parameter as well 
and returns a list of `ScoredMove` objects, which contains an entry for each possible next move.

In [ ]:
import chess.engine
import logging


class Engine():
    """Common interface for all chess engines"""

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        logging.error("Not implemented by this engine")

    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        logging.error("Not implemented by this engine")

Additionally, we enhance the `chess.engine.Score` class and its deriving classes to simplify code later. Many of the algorithms will need scores as bounds that cannot actually be reached. Therefore, two classes `HighestScoreType` and `LowestScoreType` are defined as well as concrete instanced of these classes `HighestScore` and `LowestScore`, respectively. `HighestScore` will be an upper bound for all possible scores and `LowestScore` a lower bound. The method `_score_tuple` is responsible for returning a tuple that will be used for ordering and therefore needs to be overwritten by both classes. Additionally, there are a few abstract methods that need to be implemented. 

In [ ]:
from chess.engine import Score, Cp
from typing import Tuple, Optional
import math


class HighestScoreType(Score):

    def _score_tuple(self) -> Tuple[bool, bool, bool, int, Optional[int]]:
        return (True, True, True, math.inf, math.inf)

    def mate(self) -> None:
        return None

    def score(self, *, mate_score: Optional[int] = None) -> int:
        raise NotImplementedError

    def wdl(self, *, model="sf", ply: int = 30):
        raise NotImplementedError

    def __neg__(self) -> Cp:
        return LowestScoreType()

    def __pos__(self) -> Cp:
        raise NotImplementedError

    def __abs__(self) -> Cp:
        raise NotImplementedError


HighestScore = HighestScoreType()


class LowestScoreType(Score):

    def _score_tuple(self) -> Tuple[bool, bool, bool, int, Optional[int]]:
        return (False, False, False, -math.inf, math.inf)

    def mate(self) -> None:
        return None

    def score(self, *, mate_score: Optional[int] = None) -> int:
        raise NotImplementedError

    def wdl(self, *, model="sf", ply: int = 30):
        raise NotImplementedError

    def __neg__(self) -> Cp:
        return HighestScoreType()

    def __pos__(self) -> Cp:
        raise NotImplementedError

    def __abs__(self) -> Cp:
        raise NotImplementedError


LowestScore = LowestScoreType()

We can compare different scores and see that the `LowestScore` and `HighestScore` in deed work as bounds.

In [ ]:
from chess.engine import Mate, Cp, MateGiven

LowestScore < Mate(-0) < Mate(-1) < Cp(-50) < Cp(200) < Mate(4) < Mate(
    1
) < MateGiven < HighestScore

Another required functionality that `python-chess` does not provide is the ability to increase the number of moves of a Mate. We define a method `increased_mate` that will be added to the `chess.engine.Mate` class. It takes a number `move_increase` as a parameter and returns a mate with the number of moves increased by `move_increase`. A similar function is written for the `MateGivenType`, which will return a mate with the given number.

In [ ]:
import chess.engine
from chess.engine import Mate


def increased_mate(self, move_increase: int) -> Mate:
    if self > Cp(0):
        return Mate(self.mate() + move_increase)
    return Mate(self.mate() - move_increase)


chess.engine.Mate.increased_mate = increased_mate


def increased_mate(self, move_increase: int) -> Mate:
    return Mate(move_increase)


chess.engine.MateGivenType.increased_mate = increased_mate

We can write some test cases for the different combinations.

In [ ]:
assert Mate(3).increased_mate(2) == Mate(5)
assert MateGiven.increased_mate(3) == Mate(3)
assert Mate(-3).increased_mate(2) == Mate(-5)
assert Mate(-0).increased_mate(3) == Mate(-3)